In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


llm = ChatOpenAI(model='gpt-3.5-turbo-0125', max_tokens=50)

prompt_text = "Generate one data similar in format to the provided examples in one line Chinese.\n{examples}"
prompt = ChatPromptTemplate.from_template(prompt_text)

parser = StrOutputParser()

chain = {"examples": RunnablePassthrough()} | prompt | llm | parser


### data1의 중국어 상품명 데이터 늘리기

[Hyperparameter 설명]
1. n_sample : 중국어 상품명 하나를 생성하기 위해 참고할 예시의 개수 (기존의 data1에서 랜덤 추출)
2. n_generate : data1을 한번 읽어들여왔을 때 새로운 데이터를 생성하는 횟수 (=늘어나는 데이터 개수)

[전체 프로세스]
1. product_name_chn(list)에 data1 불러옴
2. data1에서 n_sample개의 예시를 랜덤 추출하여 중국어 상품명 하나 생성
3. 2번 과정을 n_gerenate번 반복하고 생성된 n_generate개의 데이터를 data1에 추가

In [6]:
# Hyperparameters

n_sample = 5
n_generate = 12000

In [10]:
import random
import pandas as pd


product_name_chn = pd.read_csv("data/data1.csv")["product_name_chn"].tolist()

random_examples = []

for _ in range(n_generate):
    input = random.sample(product_name_chn, n_sample)
    # for item in input: product_name_chn.remove(item) # without replacement

    examples = ""
    for line in input: examples += (line + " \n")
    random_examples.append(examples)
        
new_product_name_chn = chain.batch(random_examples, config={"max_concurrency": 100})

new_product_name_chn = pd.Series(data=new_product_name_chn, name="product_name_chn")
new_product_name_chn.to_csv("data/data1.csv", mode='a', header=False, index=False)